# Creation of the CNN

### Importations 

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd
import time


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


In [3]:
print("Tmux a vraiment run cette schisse.")

Tmux a vraiment run cette schisse.


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [4]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset.loc[0:10]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[1.924007832111607, 3.3174552113304, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
1,0,"[3.395902870077362, 2.0557500170094034, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
2,0,"[1.526185199528145, 2.2726572315160185, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
3,0,"[2.6692549127850898, 3.4855723081023084, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
4,0,"[3.015681218831948, 1.643219467690823, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
5,0,"[1.556422618908361, 2.8311521189608246, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
6,0,"[1.616051796995994, 2.967584831240267, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
7,0,"[1.5000025446100484, 2.4977440714501156, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
8,0,"[3.4733630593992753, 2.729269174981904, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
9,0,"[1.7578897843542212, 1.829722126402657, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0


In [5]:
Dataset.iloc[1]

Room                                                              0
Array_position         [3.395902870077362, 2.0557500170094034, 1.5]
Distance                                                          1
Absorption                                                 0.702471
SNR                                                               0
Audio_file        /home/janjar/Dataset/Trainingset/audio_signals...
Phase_Matrix      /home/janjar/Dataset/Trainingset/phase_matrix/...
Label                                                             1
Name: 1, dtype: object

In [6]:
Dataset.loc[0:10]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[1.924007832111607, 3.3174552113304, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
1,0,"[3.395902870077362, 2.0557500170094034, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
2,0,"[1.526185199528145, 2.2726572315160185, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
3,0,"[2.6692549127850898, 3.4855723081023084, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
4,0,"[3.015681218831948, 1.643219467690823, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
5,0,"[1.556422618908361, 2.8311521189608246, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
6,0,"[1.616051796995994, 2.967584831240267, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
7,0,"[1.5000025446100484, 2.4977440714501156, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
8,0,"[3.4733630593992753, 2.729269174981904, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
9,0,"[1.7578897843542212, 1.829722126402657, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0


#### Defining our number of samples.

In [7]:
number_of_rooms = Dataset.shape[0]

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [8]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,4,129,390])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    #print(fileName_matrix)
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(140, 4, 129, 390)

In [9]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,4,129,390)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([140, 4, 129, 390])

In [52]:
final_data = torch.ones([140*390, 4,129], dtype=torch.float64)

In [53]:
i = 0
for setup in range(140):
     for nb_tf in range(390):
        final_data[i] = data_matrix[setup,:,:,nb_tf]
        i = i+1
    

In [54]:
final_data.shape

torch.Size([54600, 4, 129])

### Function that reshape a label into the right format for the data_targets.

In [55]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [56]:
data_targets = torch.zeros([number_of_rooms], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = Dataset.iloc[i]['Label']
data_targets.shape   

torch.Size([140])

In [57]:
data_targets[0]

tensor(1., dtype=torch.float64)

In [58]:
final_targets = torch.ones([140*390], dtype=torch.float64)
test = []
for i in range(140):
    curr = [data_targets[i].item()]*390
    test.append(curr)
    

In [59]:
final_targets = torch.FloatTensor(test)
final_targets = final_targets.flatten()
final_targets.shape

torch.Size([54600])

In [60]:
final_targets[780:1170]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

# Architecture 1.0 : vanilla CNN for DOA 

 In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the   sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [19]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 4):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(390096, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(nb_hidden*4,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,2)
        
    
    def forward(self, x, n):
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,0,:,:].view(-1,1,129,390)
        #print("Shape of test:",test.shape)
        output = torch.zeros([390096,1])
        for i in range(n):
            
            y = x[:,:,i,:,:].view(-1,1,129,390)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 390096)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        output = F.relu(self.fc2(output))
        output = self.dropout(output)
        output = F.softmax(self.fc3(output))        

        return output            

In [20]:
class ws_Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 390 ):
        super(ws_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(1008, nb_hidden)# (1x1008) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden*n,512)
        self.fc3 = nn.Linear(512,2)
        
    
    def forward(self, x, n):
        
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,:,:,0].view(-1,1,4,129)
        #print("Shape of test:",test.shape)
        output = torch.zeros([1008,1])
        
        for i in range(n):
            y = x[:,:,:,:,i].view(-1,1,4,129)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 1008)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        #print("Shape of the out of fc1:",output.shape)        
                
        output = F.relu(self.fc2(output))
        
        #print("Shape of the out of fc2:",output.shape) 

        output = F.softmax(self.fc3(output))   
        
        return output  

In [104]:
class SimpleModel(nn.Module):

    def __init__(self, nb_hidden=512):
        super(SimpleModel, self).__init__()
        self.cl1 = nn.Conv2d(1, 64, kernel_size=2,stride=1)
        self.cl2 = nn.Conv2d(64, 64, kernel_size=2,stride=1)
        self.cl3 = nn.Conv2d(64, 64, kernel_size=2,stride=1)
        self.fc1 = nn.Linear(8064, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,2)
 
 
    def forward(self, x):
       
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.cl3(x))
        x = F.relu(self.fc1(x.view(-1, 8064)))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

    def compute_nb_errors(self,model, data_input, data_target, mini_batch_size):

        nb_data_errors = 0
        for b in range(0, data_input.size(0), mini_batch_size):
            a = model(data_input.narrow(0, b, mini_batch_size))
            val = torch.max(a,1)[1]
            for k in range(mini_batch_size):
                if data_target.data[b + k] != val[k]:
                    nb_data_errors = nb_data_errors + 1

        return nb_data_errors

### Preparing the Data

In [105]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,1,4,129)
    return new_train_input

In [106]:
def prep_input_model2(train_input):
    new_train_input = train_input.view(-1,1,4,129,390)
    return new_train_input

In [107]:
train_data = prep_input_vanilla(final_data)
train_data.shape

torch.Size([54600, 1, 4, 129])

train_data = prep_input_model2(data_matrix)
train_data.shape

In [108]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1)
    return new_train_input

In [109]:
train_targets = prep_labels_vanilla(final_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_data, train_targets = Variable(train_data), Variable(train_targets)
train_data.shape
train_targets.shape

torch.Size([54600])

### Spliting the Training Data into Training/Validation.

In [110]:
training_data = train_data[0:50000]
training_targets = train_targets[0:50000]
validation_data = train_data[50000:54600]
validation_targets = train_targets[50000:54600]
training_data.shape, validation_data.shape, training_targets.shape, validation_targets.shape,len(training_targets),len(validation_targets)

(torch.Size([50000, 1, 4, 129]),
 torch.Size([4600, 1, 4, 129]),
 torch.Size([50000]),
 torch.Size([4600]),
 50000,
 4600)

In [111]:
train_targets

tensor([1, 1, 1,  ..., 0, 0, 0])

### End of Sanity check.

## 1.2 Training the model

In [112]:
model = Net()

In [113]:
model1 = ws_Net()

In [114]:
model2 = (SimpleModel()).float()

In [115]:
mini_batch_size = 10
nb_epochs = 10
eta = 0.01 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
optimizer1 = torch.optim.Adam(model.parameters(), lr = eta, weight_decay=0.001) #ADAM
net_ch = 4
ws_ch = 390

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [116]:
def train_model_3(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size,n):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model(train_input.narrow(0, b, mini_batch_size),n)
            target = train_target.narrow(0, b, mini_batch_size)
            indices = output.argmax(1)
            print("batch :",b," Output :",output,"Targets:", target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

    

In [117]:
def train_simple_model(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model((train_input.narrow(0, b, mini_batch_size)).float())
            target = train_target.narrow(0, b, mini_batch_size)
            indices = output.argmax(1)
            #print("batch :",b," Output :",output,"Targets:", target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

In [118]:
train_simple_model(model2,optimizer1,nb_epochs,training_data,training_targets,mini_batch_size)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time for 1 epochs is :72.02747130393982
Loss for epoch  0 is 0.684106707572937 
Loss for epoch  1 is 0.684106707572937 
Loss for epoch  2 is 0.684106707572937 
Loss for epoch  3 is 0.684106707572937 
Loss for epoch  4 is 0.684106707572937 


KeyboardInterrupt: 

In [119]:
train_model_3(model,optimizer1,nb_epochs,training_data,training_targets,mini_batch_size,net_ch)

IndexError: too many indices for tensor of dimension 4

In [126]:
t = model(validation_data[10:20],4)
t

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.5331, 0.4669],
        [0.5286, 0.4714],
        [0.5245, 0.4755],
        [0.5264, 0.4736],
        [0.5281, 0.4719],
        [0.5284, 0.4716],
        [0.5246, 0.4754],
        [0.5336, 0.4664],
        [0.5213, 0.4787],
        [0.5343, 0.4657]], grad_fn=<SoftmaxBackward>)

In [71]:
validation_targets[10:20]

tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [125]:
train_model_3(model1,optimizer1,nb_epochs,training_data,training_targets,mini_batch_size,ws_ch)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114, 0.4886],
        [0.5147, 0.4853]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.5154, 0.4846],
        [0.5128, 0.4872],
        [0.5108, 0.4892],
        [0.5109, 0.4891],
        [0.5123, 0.4877],
        [0.5158, 0.4842],
        [0.5153, 0.4847],
        [0.5151, 0.4849],
        [0.5146, 0.4854],
        [0.5148, 0.4852]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])
batch : 20  Output : tensor([[0.5152, 0.4848],
        [0.5166, 0.4834],
        [0.5146, 0.4854],
        [0.5153, 0.4847],
        [0.5168, 0.4832],
        [0.5172, 0.4828],
        [0.5140, 0.4860],
        [0.5166, 0.4834],
        [0.5173, 0.4827],
        [0.5128, 0.4872]], grad_

batch : 110  Output : tensor([[0.5157, 0.4843],
        [0.5119, 0.4881],
        [0.5134, 0.4866],
        [0.5175, 0.4825],
        [0.5140, 0.4860],
        [0.5127, 0.4873],
        [0.5130, 0.4870],
        [0.5163, 0.4837],
        [0.5098, 0.4902],
        [0.5159, 0.4841]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  1 is 0.6912064552307129 
batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114, 0.4886],
        [0.5147, 0.4853]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.5154, 0.4846],
        [0.5128, 0.4872],
        [0.5108, 0.4892],
        [0.5109, 0.4891],
        [0.5123, 0.4877],
        [0.5158, 0.4842],
        [0.5153, 0.4847],
        [0.5151, 0.4849],
        [0.5146,

batch : 100  Output : tensor([[0.5158, 0.4842],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5162, 0.4838],
        [0.5144, 0.4856],
        [0.5086, 0.4914],
        [0.5165, 0.4835],
        [0.5120, 0.4880],
        [0.5161, 0.4839],
        [0.5141, 0.4859]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
batch : 110  Output : tensor([[0.5157, 0.4843],
        [0.5119, 0.4881],
        [0.5134, 0.4866],
        [0.5175, 0.4825],
        [0.5140, 0.4860],
        [0.5127, 0.4873],
        [0.5130, 0.4870],
        [0.5163, 0.4837],
        [0.5098, 0.4902],
        [0.5159, 0.4841]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  3 is 0.6912064552307129 
batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114

KeyboardInterrupt: 